# Семинар 1. Предобработка текстовых данных

<img src="data/imgs/photo-1472737817652-4120ab61af6c.jpeg">

Компьютеры не понимают человеческие языки, поэтому текстовые данные нужно каким-то образом преобразовывать. Это часто не простой и не тривиальный процесс. На этом занятии мы разберем основные способы очистки данных, нормализации и векторизации. Также мы посмотрим на готовые интрументы, предназначенные для работы с русским языком. В конце занятия мы попробуем порешать задачу автоматического определения тональности твитов.

## Очистка данных

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. От всего этого нужно избавляться.

In [ ]:
# сразу импортируем все нужные библиотеки
# подробнее о каждой из них я расскажу по ходу
import string
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re
mystem = Mystem()
morph = MorphAnalyzer()

<font color=red>Ошибки:</font>
Если возникает ошибка **ImportError**, значит у вас не установлен какой-то пакет.
выполните в консоли *"pip install package_name"* (или pip3, если у вас две разные версии питона).
Также это можно сделать и внутри ноутбука, просто поставьте **!** перед командой

In [ ]:
!pip3 install nltk pymystem3 pymorphy2

Если ругается **nltk**, нужно запустить *nltk.download()* и скачать нужные модели или данные.

In [ ]:
import nltk
nltk.download()

Возьмем в качестве примера несколько статей с Хабрахабра. Они были скачаны автоматически и в них остались некоторые тэги.

In [ ]:
# загрузим статьи в список
habr_texts = [open('data/habr/habr_{}.txt'.format(i)).read() for i in range(5)]

In [ ]:
# посмотрим на статью
print(habr_texts[1])

В html все тэги заключаются в угловые скобки. Мы можем использовать это, чтобы легко избавиться от всех тэгов сразу. Напишем простую регулярку, которая будет удалять всё, что попадает между символами **<** и **>** и не является **>**.

In [ ]:
# re - модуль регулярных выражений в питоне
# функция sub заменяет все, что подходит под шаблон, на указанный текст
def remove_tags_1(text):
    return re.sub(r'<[^>]+>', '', text)

Проверим как работает наша функция.

In [ ]:
print(remove_tags_1(habr_texts[1]))

Можно заметить, что в некоторых местах удаление тэгов приводит к тому, что между точкой и началом следующего 
предложения нет пробела, а это может помешать правильно токенизировать текст,
поэтому сделаем так, чтобы тэг заменялся пробелом.

In [ ]:
def remove_tags_2(text):
    return re.sub(r'<[^>]+>', ' ', text)

In [ ]:
print(remove_tags_2(habr_texts[0]))

Теперь куски текста не слипаются, но появились последовательности из нескольких пробелов, чтобы убрать их, добавим ещё одно регулярное выражение и применим его к тексту, из которого уже удалили тэги.

In [ ]:
def remove_tags_3(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text)
    return no_space_sequences_text

In [ ]:
print(remove_tags_3(habr_texts[3]))

Теперь текст более менее чистый.

## Токенизация

Из текста теперь нужно выделить интересующие нас объекты. Чаще всего нужно просто слова.

Самый простой способ поделить текст на слова — стандартный питоновский ***str.split*** метод.
    По умолчанию он разбивает текст по последовательностям пробелов
 (т.е. даже со второй версией remove_tags всё бы хорошо разделилось).

Попробуем на какой-нибудь очищенной статье.

In [ ]:
clean_habr_texts = [remove_tags_2(text) for text in habr_texts]

In [ ]:
clean_habr_texts[1].split()

Большая часть слов отделяется, но знаки препинания лепятся к словам.
Можно пройтись по всем словам и убрать из них пунктцацию с методом str.translate.

In [ ]:
#основные знаки преминания хранятся в питоноском модуле string.punctuation
string.punctuation

In [ ]:
# сделаем словарь, который будет всем знакам препинания сопостовлять None
# и преобразуем текст
table = str.maketrans({ch: None for ch in string.punctuation})
[word.translate(table) for word in clean_habr_texts[3].split()][:10]

Все разобралось очень хорошо, остались слова в кавычках-елочках и лапках, длинное тире, но наверное это и к лучшему.
Не к лучшему то, что многоточие не удаляется.
Ещё можно отметить то, что такой способ будет, например, удалять апострофы.

In [ ]:
"father's".translate(table)

Исправить и то и другое можно добавив или удалив какие-то знаки пунктуации к/из ***string.punctuation***.

In [ ]:
punct_extended = string.punctuation + '«»—…“”'
table = str.maketrans({ch: None for ch in punct_extended})
[word.translate(table) for word in clean_habr_texts[3].split()][:10]

In [ ]:
punct_no_apostrophie = re.sub("'", '', string.punctuation)
table = str.maketrans({ch: None for ch in punct_no_apostrophie})
print("father's".translate(table))

Ещё есть готовые токенизаторы из nltk. Они не удаляют пунктуацию, а выделяют её отдельным токеном

Например ***wordpunct_tokenizer*** разбирает по регулярке - '**\w+|[^\w\s]+**'

In [ ]:
wordpunct_tokenize(clean_habr_texts[1])

Ещё есть ***word_tokenize***. Он также построен на регулярках, но они там более сложные (учитывается последовательность некоторых 
символов, символы начала, конца слова и т.д). 

Специально подобранного под русский язык токенизатора там нет, 
но и с английским всё работает достаточно хорошо:
сокращения типа **т.к** собираются в один токен, дефисные слова тоже не разделяются. Многоточия тут тоже не отделяются, но это можно поправить.

In [ ]:
word_tokenize(clean_habr_texts[3])

# Нормализация

Часто после выделения слов (или других вещей) нужно привести разные формы одного слова к одной стандартной или получить подробную информацию об отдельном слове.

Эта задача уже в разы сложнее предыдущих. Простой функцией тут не отделаешься. К счастью, есть готовые инструменты.

Для русского основых варианта два: Mystem и Pymorphy.

Майстем работает немного лучше и сам токенизирует,
поэтому можно в него засовывать сырой текст.

In [ ]:
# mystem.lemmatize функция лемматизации в майстеме
# сам объект mystem нужно заранее инициализировать
# мы сделали это в начале тетрадки строчкой "mystem = Mystem()"
mystem.lemmatize(clean_habr_texts[1])[:10]

In [ ]:
# Если нужна грамматическая информация или надо сохранить ненормализованный текст,
# есть функция mystem.analyze
words_analized = mystem.analyze(clean_habr_texts[1])

In [ ]:
# возвращает она список словарей
# каждый словарь имеет либо одно поле 'text' (когда попался пробел) или text и analysis
# в analysis снова список словарей с вариантами разбора (первый самый вероятный)
# поля в analysis - 'gr' - грамматическая информация, 'lex' - лемма
# analysis - может быть пустым списком
words_analized[:10]

In [ ]:
print('Слово - ', words_analized[0]['text'])
print('Разбор слова - ', words_analized[0]['analysis'][0])
print('Лемма слова - ', words_analized[0]['analysis'][0]['lex'])
print('Грамматическая информация слова - ', words_analized[0]['analysis'][0]['gr'])

In [ ]:
#леммы можно достать в одну строчку
[parse['analysis'][0]['lex'] for parse in words_analized if parse.get('analysis')][:10]

Недостатки Mystem: это продукт Яндекса с некоторыми ограничениями на использование, больше он не развивается.

Pymorphy - открытый и развивается (можно поучаствовать на гитхабе)

Ссылка на репозиторий: https://github.com/kmike/pymorphy2

У него нет втстроенной токенизации и он расценивает всё как слово (это одновременно плюс и минус).

In [ ]:
# основная функция - pymorphy.parse
words_analized = [morph.parse(token) for token in word_tokenize(clean_habr_texts[2])]

In [ ]:
# Она похожа на analyze в майстеме только возрващает список объектов Parse
# Первый в списке - самый вероятный разбор (у каждого есть score)
# Информация достается через атрибут (Parse.word - например)
# Грамматическая информация хранится в объекте OpencorporaTag и из него удобно доставать
# части речи или другие категории
print('Cлово - ', words_analized[0][0].word)
print('Разбор слова - ', words_analized[0][0])
print('Лемма слова - ', words_analized[0][0].normal_form)
print('Грамматическая информация слова - ', words_analized[0][0].tag)
print('Часть речи слова - ', words_analized[0][0].tag.POS)
print('Род слова - ', words_analized[0][0].tag.gender)
print('Число  слова - ', words_analized[0][0].tag.number)
print('Падеж слова - ', words_analized[0][0].tag.case)

## Дополнительная очистка текста

Пунктуация часто совсем не нужна и поэтому можно выбросить её заранее. Если нужно обрабатывать много текста, это может немного ускорить процесс.


In [ ]:
# Оставим только буквено-численные токены
# Это не самый лучший вариант, так как удалятся сокращения с точкой, слова через дефис
text = 'В этом случае слова вроде т.к. и по-другому не пройдут фильтр и будут удалены.'
good_tokens = [word for word in word_tokenize(text) if word.isalnum()]
[morph.parse(token)[0].normal_form for token in good_tokens]

In [ ]:
#можно сделать фильтр по длине, или оставить всё, что не целиком состоит из знаков препинания
good_tokens = [word for word in word_tokenize(text) if len(word) > 1 
                                                    and not all([ch in punct_extended for ch in word])]
[morph.parse(token)[0].normal_form for token in good_tokens]

Если не хочется делать нормализацию, то можно хотя бы просто привести все слова к одному регистру.

In [ ]:
print('СпАсИбО'.lower())
print('СпАсИбО'.upper())
print('СпАсИбО'.capitalize())

После нормализации можно убрать стоп-слова (предлоги, союзы, местоимения, частотные слова).

In [ ]:
# стоп-слова есть в nltk
stops = stopwords.words('russian')
print(stops)

Список не идеальный и его можно расширять под свои задачи (в примере можно бы ещё удалить слово "это" и "вроде")

In [ ]:
words_normalized = [morph.parse(token)[0].normal_form for token in good_tokens]
words_normalized_no_stops = [word for word in words_normalized if word not in stops]
print(words_normalized_no_stops)

# Классификация

Попробуем теперь применить эти знания на настоящей задаче. Возьмем данные Dialog Evaluation по анализу тональности твитов пользователей в адрес телекоммуникационных компаний. В изначальной формулировке нужно определять тональность относительно упомянутой компании, но мы упростим её до простого определения тональности твита.

Таким образом нам нужно по тексту твита приписать ему класс - негативный, нейтральный или положительный.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
%matplotlib inline  
pd.set_option('max_colwidth', 1000)

In [ ]:
# я заранее подготовил для вас данные
train_data = pd.read_csv('data/sentiment_twitter/train_sentiment_ttk.tsv', sep='\t')
test_data = pd.read_csv('data/sentiment_twitter/test_sentiment_ttk.tsv', sep='\t')

Посмотрим на данные, чтобы убедиться, что всё нормально

In [ ]:
train_data.head()

In [ ]:
test_data.head()

Проверим распределение классов (-1 - негативный отзыв, 1 - положительный, 0 - отрицательный)

In [ ]:
train_data.label.hist()

In [ ]:
test_data.label.hist()

Видно, что положительный заметно меньше. 

Посмотрим на какой-нибудь негативный твит.

In [ ]:
train_data['text'][10]

И на положительный

In [ ]:
train_data['text'][9]

## Бейзлайн

В начале стоит попробовать самый простой возможный вариант. 
Count_vectorizer - без какой-либо нормализации подойдет.

Почти у всего в sklearn индентичный интерфейс. 
1. Нужно инициализировать нужный объект

    ***`vectorizer = CountVectorizer()`***

2. "Обучить" модель на наших данных.
    
    ***`vectorizer.fit(texts)`***
    
3. Преобразовать с помощью обученной модели наши данные в вектора.
    
    ***`X = vectorizer.transform(texts)`***

In [ ]:
# посмотрим какие опции есть у count_vectorizer
?CountVectorizer

In [ ]:
# у count_vectorizer есть встроенный токенизатор, поэтому можно подавать текст напрямую
# обучим векторайзер на обучающей выборке и преобразуем тексты в векторы
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train_data.text.values) 

X_train = count_vectorizer.transform(train_data.text.values)
X_test = count_vectorizer.transform(test_data.text.values)

In [ ]:
# посмотрим на размеры матрицы
# первое число - количество твитов
# второе - размер каждого вектора (равен размеру словаря)
X_train.shape

In [ ]:
# нужно чтобы второе число совпадало в обучающей и тестовой выборке
X_test.shape

In [ ]:
# выделим классы в отдельные переменные
y_train = train_data.label.values
y_test = test_data.label.values

В качесте классификатора будем использовать Логистическую регрессию.

Интерфейс почти в точности такой же. 
1. Нужно инициализировать нужный объект

    ***`clf = LogisticRegression()`***

2. Обучить модель на наших заранее преобразованных данных.
    
    ***`clf.fit(X)`***
    
3. Предсказать классы на новых данных.
    
    ***`preds = clf.predict(texts)`***

In [ ]:
#также можно для начала посмотреть параметры
#основной параметр это С, коэффициент регуляризации.
#Регуляризация нужна для того, чтобы ограничить значения выучиваемых параметров, чтобы избежать переобучения
# l2 стоит по умолчанию и обычно лучше работает, но
# l1 зануляет ненужные признаки и увеличивает значения важных
# поэтому её можно использовать для отбора признаков
#попробуйте перебрать значения (0.01, 0.1, 1(по умолчанию), 10, 100)
?LogisticRegression

In [ ]:
clf = LogisticRegression(penalty="l1", C=0.1)
clf.fit(X_train, y_train)

In [ ]:
# предскажем значения тестовых твитов
y_pred = clf.predict(X_test)

In [ ]:
# замерим качество классификации
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

У некоторых классификаторов можно посмотреть значимость признаков. У логрега они хранится в ***clf.coef_*** . Это массив размером (количество классов, количество признаков). 

Признаки можно достать из векторайзера с помощью метода ***get_feature_names***.

Вместе их можно соотнести встроеной функцией *zip*. При использовании **L1** регуляризации значений признака можно интерпретировать как важность.

In [ ]:
def print_important(vectorizer, clf, topn=10):
    features = vectorizer.get_feature_names()
    classes = clf.classes_
    importances = clf.coef_
    for i, cls in enumerate(classes):
        print('Значимые слова для класса - ', cls)
        important_words = sorted(list(zip(features, importances[i])), key=lambda x: abs(x[1]), reverse=True)[:topn]
        print([word for word,_ in important_words])
        print()


In [ ]:
print_important(count_vectorizer, clf)

Для визуализации важных слов можно ещё использовать библиотеку **wordcloud**.

In [ ]:
#!pip install wordcloud

In [ ]:
from wordcloud import WordCloud

In [ ]:
top = 150
features = count_vectorizer.get_feature_names()
importances = clf.coef_
classes = clf.classes_
words_with_weights = sorted(list(zip(features, importances[0])),key=lambda x: abs(x[1]), reverse=True)
only_words = [word for word,_ in words_with_weights][:top]

cloud = WordCloud(width=1000, height=500).generate(' '.join(only_words))
plt.figure(figsize=(15, 15))
plt.imshow(cloud)
plt.axis('off')

### Попробуем теперь TfidfVectorizer.

In [ ]:
# интерфейс точно такой же, но немного отличаются параметры
# токенизация по умолчанию также включена, поэтому подаём текст как есть
?TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data.text.values)
X_train = tfidf.transform(train_data.text.values)
X_test = tfidf.transform(test_data.text.values)

In [ ]:
X_train.shape

In [ ]:
clf = LogisticRegression(penalty='l1')
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# замерим качество классификации
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))

Также бывает полезно посмотреть на confision matrix.

Для красоты визуализируем с помощью библиотеки seaborn.

In [ ]:
#!pip install seaborn

In [ ]:
import seaborn as sns

In [ ]:
labels = clf.classes_
fig, ax = plt.subplots(figsize=(10,5))
sns.heatmap(data=confusion_matrix(y_test, y_pred), annot=True, 
            fmt="d", xticklabels=labels, yticklabels=labels, ax=ax)
plt.title("Confusion matrix")
plt.show()

In [ ]:
# если не устанавливается, посмотрите просто так
confusion_matrix(y_test, y_pred)

In [ ]:
print_important(tfidf, clf, 20)

## Улучшение качества классификации

#### По самым характерным словам видно, что в корпусе есть мусор, стоп слова, разные формы одного слова. Чтобы улучшить результат попробуйте почистить данные и добавить нормализацию.

In [ ]:
def normalize(text):
    """
    функция нормализации
    
    ::парметры::
    @text - ненормализованный текст (string)
    
    ::returns::
    нормализованный текст (string)
    """
    
    tokens = # ваш код здесь
    lemmas = # ваш код здесь
    
    return ' '.join(lemmas)
    

Чтобы применить нормализацию ко всему корпусу, воспользуйтесь функцией apply.

In [ ]:
train_data['normalized'] = train_data['text'].apply(normalize)
test_data['normalized'] = test_data['text'].apply(normalize)

Обучите новую модель на нормализованных данных.

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(train_data['normalized'].values)

X_train = tfidf.transform(train_data['normalized'].values)
X_test = tfidf.transform(test_data['normalized'].values)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))
print('Макросредняя F1 мера - ',f1_score(y_test, y_pred, average='macro'))
print('Микросредняя F1 мера - ',f1_score(y_test, y_pred, average='micro'))